In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [251]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None, verbose = False):
        
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.model_name = 'Linear Regression'

        # store the variables in object scope
        self.alpha: float = alpha
        self.num_iter: int = num_iter
        self.early_stop: float = early_stop
        self.intercept: bool = intercept
        self.init_weight = init_weight  ### For testing correctness.
        
        self.verbose = verbose
        
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.
                
                
                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """
        
        # i'm not sure what the dimensions of the input data mean
        # since they say add a column, i'm assuming that each row is an example
        
        self.X = np.array(X_train)
        self.y = np.array(y_train).reshape(-1, 1) 
        
        if self.intercept:
            ones = np.ones((self.X.shape[0], 1))
            self.X = np.hstack([ones, self.X])
            self.log(self.X.shape)
            self.log(self.X)
            
        np.random.seed(16)
        self.coef = np.random.uniform(-1, 1, (self.X.shape[1], 1),)
        self.log(self.coef)
        
        self.gradient_descent()
        
        # self.coef = self.init_weight #### Please change this after you get the example right.
        
    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.
            
            TODO: 5. Think about the matrix format of the gradient of the loss function.
        """
        
        # SE Loss function. Ignoring the normalization across examples. Feels strange, but I guess that it should still work. Depending on the dataset, could approach overflow. 
        # I would rather think of this in terms of forward and backward pass
        # The way that things are separted into two function calls where the forward pass values can't be passed into the backward pass values annoys me. But w.e. this is how the assignment is designed. 
        
        # forward pass
        # pred = X*coef
        # diff = y-pred
        # loss = sum (y-pred)**2
       
        # print(self.y.shape)
        # print(self.X.shape)
        # print(self.coef.shape)
        
        # backward pass (gradients) 
        d_loss = 1
        d_diff = d_loss * 2 * (self.y - self.X @ self.coef)
        d_pred = d_diff * -1
        # each coefficient affects every prediction
        # each coefficient affects a column of X. an increase in an coefficient leads to an proportionate increase in the predicted values of that column of X. 
        # d_pred = (num_examples)
        # manual backward pass. thinking about how each value affects the subsequent values in the graph
        d_coef = (np.asarray(d_pred) * np.asarray(self.X)).sum(axis=0)
        
        # doesn't make sense to have to calculate these values twice
        # can i just keep the gradient inside the gradient_descent function
        grad_coef = (-2 * (self.y - self.X @ self.coef) * self.X).sum(axis=0)
        
        print(np.isclose(d_coef, grad_coef))
        
        self.grad_coef = grad_coef
        
    def gradient_descent(self):
        
        """
            Training function
            
            TODO: 6. Calculate the loss with current coefficients.
                  7. Update the temp_coef with learning rate and gradient.
                  8. Calculate the loss with temp_coef.
                  9. Implement the self adeptive learning rate. 
                      a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                         And update coef, with temp_coef.
                      b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                         Don't update coef.
                  10. Add the loss to loss list we create.
        """
        
        self.loss = []
        
        for i in range(self.num_iter):
            
            # forward pass
            preds = self.X @ self.coef
            errors = self.y - preds
            squared_errors = (errors**2)
            loss = squared_errors.sum()
            
            self.loss.append(loss)
            
            # backward pass
            dloss = 1
            dsquared_errors = dloss * np.ones_like(squared_errors)
            # dsquared_errors = dloss * 1
            derrors = dsquared_errors * 2 * errors
            dpreds = derrors * -1
            # dcoef = (dpreds * self.X).sum(axis=0) # for a given coefficients affects a column of x's, which each affect a different prediction
            dcoef = (dpreds * self.X).sum(axis=0).reshape(-1, 1) # for a given coefficients affects a column of x's, which each affect a different prediction
            # print(self.coef.shape)
            
            # if(self.coef.shape != (2, 1)):
            #     break
            # break
            
            # print("test", self.coef)
            # print(dcoef)
            # print("test1", self.alpha * dcoef)
            # print("test2", (self.coef * self.alpha).shape)
            # print("test3", np.subtract(self.coef, self.alpha * dcoef))
            # test1 = self.coef
            # test2 = self.alpha * dcoef
            # print("test4", test1 - test2)
            # print(type(self.coef), type(dcoef))
            # break

            temp_coef = self.coef - self.alpha * dcoef
            # print(self.coef.shape, temp_coef.shape, (self.alpha * dcoef).shape)
            # print(self.coef, temp_coef.shape)
            temp_loss = np.sum((self.y - self.X @ temp_coef)**2) # technically double calculates, which doesn't need to be done. i'll fix this later.
            
            pre_error = loss
            
            current_error = temp_loss
            
            ### This is the early stop, don't modify fllowing three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                print(f"EARLY STOP @ {i}")
                self.coef = temp_coef
                return self
            
            if current_error <= pre_error:
                self.alpha *= 1.3
                self.coef = temp_coef
            else:
                self.alpha *= 0.9 # don't update coefs
                
            self.loss.append(loss)
            
            if i % 10000 == 0:
                print('Iteration: ' +  str(i))
                print('Coef: '+ str(self.coef))
                print('Loss: ' + str(current_error))            
                
        return self
    
    def ind_predict(self, x: list):
        """
            Predict the value based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                result: prediction of given data point
        """
        
        """
            TODO: 11. Implement the prediction function
        """
        result = 1
        
        return result
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            X: Matrix, array or list. Input feature point.
            
            Return:
                ret: prediction of given data matrix
        """
        
        """
            TODO: 12. Make sure add the 1's column like we did to add intercept.
                  13. Revise the following for-loop to call ind_predict to get predictions.

        """
        
        ret = []
        X = np.mat(X)
        if self.intercept:
            ones = 0
            X = X
        for x in X:
            ret.append(1)
        return ret
        
    def log(self, str):
        if self.verbose == True:
            print(str)

In [252]:
def min_max_normalize(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [253]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() +  20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [238]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

Iteration: 0
Coef: [[-0.55341784]
 [ 0.04632668]]
Loss: 1.0512540537247319e+27
Iteration: 10000
Coef: [[-0.42121603]
 [30.03076879]]
Loss: 20946.45307079826
Iteration: 20000
Coef: [[-0.33277167]
 [30.03048674]]
Loss: 20764.404443420626
Iteration: 30000
Coef: [[-0.24469976]
 [30.03048024]]
Loss: 20584.9381638687
Iteration: 40000
Coef: [[-0.15701385]
 [30.03021484]]
Loss: 20406.880884395716
Iteration: 50000
Coef: [[-0.069677  ]
 [30.03008841]]
Loss: 20230.36933475199
Iteration: 60000
Coef: [[1.72549084e-02]
 [3.00300993e+01]]
Loss: 20056.33541968811
Iteration: 70000
Coef: [[ 0.10380399]
 [30.02982407]]
Loss: 19882.431533466686
Iteration: 80000
Coef: [[ 0.18998496]
 [30.0298398 ]]
Loss: 19710.67532541356
Iteration: 90000
Coef: [[ 0.2757859 ]
 [30.02956728]]
Loss: 19539.945552369667
Iteration: 100000
Coef: [[ 0.3612478 ]
 [30.02944095]]
Loss: 19370.95718696334
Iteration: 110000
Coef: [[ 0.44631426]
 [30.02945397]]
Loss: 19204.329388099533
Iteration: 120000
Coef: [[ 0.53100543]
 [30.0291806

KeyboardInterrupt: 

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [255]:
np.array(clf.predict(X))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

In [256]:
X_norm = min_max_normalize(X)
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X_norm,y)

Iteration: 0
Coef: [[-0.55341784]
 [ 0.04632668]]
Loss: 1.298640205014184e+16
EARLY STOP @ 1308


In [258]:
clf.coef

array([[   50.],
       [29850.]])

In [261]:
X.min(), X.max(), X.max() - X.min()

(1, 996, 995)

In [ ]:

# 29850 / 995 = 30 
# slope checks out

# x - min. every single point is shifted over by the minimum value. line is shifted over by the slope. 

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [262]:
from sklearn.linear_model import LinearRegression

In [267]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
X_norm = min_max_normalize(X)
y = wine.quality

In [264]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

800.6676988774323

In [265]:
lr2 = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight=None)
lr2.fit(X, y)


Iteration: 0
Coef: [[-0.55341784]
 [ 0.04632668]
 [ 0.10140291]]
Loss: 5287551503614576.0
Iteration: 10000
Coef: [[0.22566935]
 [0.83438408]
 [0.43902616]]
Loss: 816.584217720002
Iteration: 20000
Coef: [[0.48625587]
 [1.10121317]
 [0.38829796]]
Loss: 806.863747729821
Iteration: 30000
Coef: [[0.57608994]
 [1.19622891]
 [0.3710301 ]]
Loss: 805.6710149577236
Iteration: 40000
Coef: [[0.60605507]
 [1.23100601]
 [0.36485609]]
Loss: 805.524723167701
Iteration: 50000
Coef: [[0.61504279]
 [1.24467339]
 [0.36267951]]
Loss: 805.5062096751012
Iteration: 60000
Coef: [[0.61669178]
 [1.25095649]
 [0.3619387 ]]
Loss: 805.5033619800577
Iteration: 70000
Coef: [[0.6157666 ]
 [1.25464965]
 [0.36167055]]
Loss: 805.5024306852986
Iteration: 80000
Coef: [[0.6139414 ]
 [1.25743687]
 [0.36158585]]
Loss: 805.5017347855267
Iteration: 90000
Coef: [[0.61179927]
 [1.25990672]
 [0.36155593]]
Loss: 805.5010687469071
Iteration: 100000
Coef: [[0.60954709]
 [1.26226356]
 [0.36154123]]
Loss: 805.5004057997098
Iteration: 1

KeyboardInterrupt: 

In [268]:
lr2.fit(X_norm, y)

Iteration: 0
Coef: [[-0.42268462]
 [ 0.11000215]
 [ 0.14338641]]
Loss: 57822.871677132076
EARLY STOP @ 3624


In [ ]:
# just fucking instantaneous once again
# that's actually kinda insane to me
# i'm not sure why...


#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

In [ ]:
sum((clf.predict(X) - y)**2)

In [235]:
## Tests 

test = Linear_Regression(intercept=True, verbose=True, alpha=1e-3)
X2 = np.array([[1], [2], [3]])
y2 = np.array([1, 2, 3])
X2, y2
test.fit(X2, y2)
test.coef

(3, 2)
[[1. 1.]
 [1. 2.]
 [1. 3.]]
[[-0.55341784]
 [ 0.04632668]]
Iteration: 0
Coef: [[-0.53865325]
 [ 0.07967055]]
Loss: 18.677391590518212
EARLY STOP


array([[-3.33083181e-15],
       [ 1.00000000e+00]])